In [1]:
import sys
sys.path.append("../src/")
from loss_function import loss_function
from loss_function import cs
from torch.autograd import Variable
from evaluate import Evaluation
from lstm import LSTM
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn
import random
import time

In [2]:
import data_reader as dr

In [3]:
corpus_path = "../data/text_tokenized.txt.gz"
corpus = dr.read_corpus(corpus_path)

In [4]:
#embedding_path = "../data/vectors_pruned.200.txt.gz"
#embedding_path = "../data/glove.6B.200d.txt.gz"
embedding_path = "../data/glove.combined.300d.txt.gz"
embedding_tensor, word_to_indx = dr.getEmbeddingTensor(embedding_path)

In [5]:
ids_corpus = dr.map_corpus(corpus, word_to_indx, kernel_width = 1)

In [6]:
train_path = "../data/train_random.txt"
train = dr.read_annotations(train_path)

In [7]:
train_ex = dr.create_train_set(ids_corpus, train)

In [8]:
def normalize(x, dim):
    l2 = torch.norm(x, 2, dim)#.expand_as(x)
    l2 = torch.unsqueeze(l2, 2)
    l2 = l2.expand_as(x)
    return x / l2.clamp(min = 1e-8)

def mean_pool_pad(x, mask):
    mask 

In [9]:
import torch.utils.data

def train_model(train_data, dev_data, model):
    model.cuda()
    #model.cpu()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0004, weight_decay = 1e-5)
    model.train()

    lasttime = time.time()
    for epoch in range(1, 31):
        print("-------------\nEpoch {}:\n".format(epoch))

        loss = run_epoch(train_data, True, model, optimizer)
        #return loss
        print('Train loss: {:.6f}'.format(loss))
        torch.save(model, "model_epoch{}".format(0 + epoch))
        
        (MAP, MRR, P1, P5) = run_epoch(dev_data, False, model, optimizer)
        print('Val MAP: {:.6f}, MRR: {:.6f}, P1: {:.6f}, P5: {:.6f}'.format(MAP, MRR, P1, P5))
        
        print('This epoch took: {:.6f}'.format(time.time() - lasttime))
        lasttime = time.time()

        
def run_epoch(data, is_training, model, optimizer):
    '''
    Train model for one pass of train data, and return loss, acccuracy
    '''
    data_loader = torch.utils.data.DataLoader(
        data,
        batch_size=40,
        shuffle=True,
        num_workers=4,
        drop_last=False)

    losses = []

    if is_training:
        model.train()
    else:
        model.eval()

    for batch in data_loader:
        pid_title = torch.unsqueeze(Variable(batch['pid_title']), 1)
        pid_body = torch.unsqueeze(Variable(batch['pid_body']), 1)
        rest_title = Variable(batch['rest_title'])
        rest_body = Variable(batch['rest_body'])
        
        pid_title_pad = torch.unsqueeze(Variable(batch['pid_title_pad']), 1)
        pid_body_pad = torch.unsqueeze(Variable(batch['pid_body_pad']), 1)
        rest_title_pad = Variable(batch['rest_title_pad'])
        rest_body_pad = Variable(batch['rest_body_pad'])
        
        pid_title, pid_body = pid_title.cuda(), pid_body.cuda()
        rest_title, rest_body = rest_title.cuda(), rest_body.cuda()
        pid_title_pad, pid_body_pad = pid_title_pad.cuda(), pid_body_pad.cuda()
        rest_title_pad, rest_body_pad = rest_title_pad.cuda(), rest_body_pad.cuda()
        
        if is_training:
            optimizer.zero_grad()
        
        pt = model(pid_title)
        pb = model(pid_body)
        rt = model(rest_title)
        rb = model(rest_body)
        
        pt = normalize(pt, 2)
        pb = normalize(pb, 2)
        rt = normalize(rt, 2)
        rb = normalize(rb, 2)
        
        # we need to take the mean pooling taking into account the padding
        # tensors are of dim batch_size x samples x output_size x (len - kernel + 1)
        # pad tensors are of dim batch_size x samples x (len - kernel + 1)
        
        pid_title_pad_ex = torch.unsqueeze(pid_title_pad, 2).expand_as(pt)
        pid_body_pad_ex = torch.unsqueeze(pid_body_pad, 2).expand_as(pb)
        rest_title_pad_ex = torch.unsqueeze(rest_title_pad, 2).expand_as(rt)
        rest_body_pad_ex = torch.unsqueeze(rest_body_pad, 2).expand_as(rb)
        
        pt = torch.sum(pt * pid_title_pad_ex, dim = 3)
        pb = torch.sum(pb * pid_body_pad_ex, dim = 3)
        rt = torch.sum(rt * rest_title_pad_ex, dim = 3)
        rb = torch.sum(rb * rest_body_pad_ex, dim = 3)

        # tensors are not of dim batch_size x samples x output_size
        # need to scale down because not all uniformly padded
        
        ptp_norm = torch.unsqueeze(torch.sum(pid_title_pad, dim = 2).clamp(min = 1), 2).expand_as(pt)
        pbp_norm = torch.unsqueeze(torch.sum(pid_body_pad, dim = 2).clamp(min = 1), 2).expand_as(pb)
        rtp_norm = torch.unsqueeze(torch.sum(rest_title_pad, dim = 2).clamp(min = 1), 2).expand_as(rt)
        rbp_norm = torch.unsqueeze(torch.sum(rest_body_pad, dim = 2).clamp(min = 1), 2).expand_as(rb)
        
        pt = pt / ptp_norm
        pb = pb / pbp_norm
        rt = rt / rtp_norm
        rb = rb / rbp_norm
        
        pid_tensor = (pt + pb)/2
        rest_tensor = (rt + rb)/2
        
        if is_training:
            dropout = nn.Dropout(p = 0.2)
            # we don't need to re-scale these on eval because its just cos sim
            pid_tensor = dropout(pid_tensor)
            rest_tensor = dropout(rest_tensor)
        
        if is_training:
            loss = loss_function(pid_tensor, rest_tensor, margin = 1.0)
            loss.backward()
            losses.append(loss.cpu().data[0])
            optimizer.step()
        else:
            expanded = pid_tensor.expand_as(rest_tensor)
            similarity = cs(expanded, rest_tensor, dim=2)
            similarity = similarity.data.cpu().numpy()
            #return similarity
            labels = batch['labels'].numpy()
            l = dr.convert(similarity, labels)
            losses.extend(l)

    # Calculate epoch level scores
    if is_training:
        avg_loss = np.mean(losses)
        return avg_loss
    else:
        e = Evaluation(losses)
        MAP = e.MAP()*100
        MRR = e.MRR()*100
        P1 = e.Precision(1)*100
        P5 = e.Precision(5)*100
        return (MAP, MRR, P1, P5)

In [10]:
model = LSTM(310, embedding_tensor, 0.2)

In [11]:
val_path = "../data/test.txt"
val = dr.read_annotations(val_path, K_neg = -1, prune_pos_cnt = -1)
val_ex = dr.create_dev_set(ids_corpus, val)

In [12]:
z = train_model(train_ex, val_ex, model)

-------------
Epoch 1:

Train loss: 0.954059
Val MAP: 56.448763, MRR: 68.384175, P1: 53.225806, P5: 42.688172
This epoch took: 270.471520
-------------
Epoch 2:

Train loss: 0.851741
Val MAP: 56.503193, MRR: 68.544435, P1: 54.301075, P5: 41.827957
This epoch took: 300.624090
-------------
Epoch 3:

Train loss: 0.807282
Val MAP: 57.388543, MRR: 69.458860, P1: 53.225806, P5: 43.870968
This epoch took: 351.887605
-------------
Epoch 4:

Train loss: 0.774704
Val MAP: 57.607602, MRR: 70.851409, P1: 56.451613, P5: 42.903226
This epoch took: 353.230862
-------------
Epoch 5:

Train loss: 0.747547
Val MAP: 57.016118, MRR: 69.266900, P1: 53.763441, P5: 43.548387
This epoch took: 375.707387
-------------
Epoch 6:

Train loss: 0.723780
Val MAP: 57.981898, MRR: 69.133700, P1: 54.838710, P5: 44.193548
This epoch took: 378.340500
-------------
Epoch 7:

Train loss: 0.701311
Val MAP: 56.591714, MRR: 68.191631, P1: 53.763441, P5: 43.333333
This epoch took: 358.313595
-------------
Epoch 8:

Train loss

In [ ]:
# current best model is model_epoch6
# MAP: 57.981898, MRR: 69.133700, P1: 54.838710, P5: 44.193548


In [16]:
# "test" on dev set
model = torch.load("model_epoch6")

val_path = "../data/dev.txt"
val = dr.read_annotations(val_path, K_neg = -1, prune_pos_cnt = -1)
val_ex = dr.create_dev_set(ids_corpus, val)

model = model.cuda()
z = run_epoch(val_ex, False, model, None)
print(z)

# test set
val_path = "../data/test.txt"
val = dr.read_annotations(val_path, K_neg = -1, prune_pos_cnt = -1)
val_ex = dr.create_dev_set(ids_corpus, val)

z = run_epoch(val_ex, False, model, None)
print(z)

(58.05577513494322, 70.60486397787984, 58.201058201058196, 45.60846560846563)
(57.98189767697066, 69.13369980645021, 54.83870967741935, 44.19354838709681)
